In [1]:
from model import DeepLOBLightning
import yaml
from pathlib import Path
import pandas as pd
import re
from utility import *

from eval import *
from data import *

ckpt_path = "checkpoints/deepLOB-epoch=05-val_acc=0.0000-v2.ckpt"

model = DeepLOBLightning.load_from_checkpoint(ckpt_path)



In [2]:
def _cast_sci(obj):
    """
    递归把符合科学计数法格式的 str 转为 float
    """
    _sci_re = re.compile(
        r"""^[+-]?            # 可选正负号
            (?:\d+\.\d*|\d*\.\d+|\d+)  # 整数或小数
            [eE][+-]?\d+$    # e/E + 指数
        """,
        re.VERBOSE,
    )

    if isinstance(obj, dict):
        return {k: _cast_sci(v) for k, v in obj.items()}
    if isinstance(obj, list):
        return [_cast_sci(v) for v in obj]
    if isinstance(obj, str) and _sci_re.match(obj):
        return float(obj)
    return obj


with open("config/config.yaml", encoding="utf-8") as f:
    cfg = yaml.safe_load(f)

cfg = _cast_sci(cfg)

model_cfg = cfg["model"]
ckpt_cfg = cfg["checkpoint"]
trainer_cfg = cfg["trainer"]

set_random_seed(cfg["seed"])

data_path = Path(cfg["data"]["path"])
files = cfg["data"]["parquet_file"]
if isinstance(files, str):
    data = pd.read_parquet(data_path / files)
elif isinstance(files, list):
    data = [pd.read_parquet(data_path / f) for f in files]
else:
    raise ValueError("cfg['data']['parquet_file'] 应该是 str 或 list[str]")

In [3]:
dm = LOBDataModule(
    train_data=data[:-1],
    test_data=data[-1],
    dm_cfg=cfg["datamodule"],
    data_cfg=cfg["data"],
    task_type=model_cfg["task_type"],
)
dm.setup()

In [4]:
ckpt_path = "checkpoints/deepLOB-vlogger.version=0-epoch=07-model.monitor_metric=0.0000.ckpt"
from model import DeepLOBLightning

model = DeepLOBLightning.load_from_checkpoint(ckpt_path)

evaluate_model(model=model, datamodule=dm, dataset="train")

evaluate_model(model=model, datamodule=dm, dataset="val")

evaluate_model(model=model, datamodule=dm)


  0%|          | 0/109 [00:00<?, ?it/s]/Users/sumen/Documents/workspace/25summer/.venv/lib/python3.13/site-packages/torch/nn/modules/conv.py:549: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/Convolution.cpp:1037.)
  return F.conv2d(
100%|██████████| 109/109 [00:22<00:00,  4.88it/s]


Pearson IC = 0.1621
Spearman IC = 0.1842
R^2 (no intercept, standard) = 0.0256
R^2 (no intercept, formula) = 0.0256
MSE = 7.352626


100%|██████████| 13/13 [00:10<00:00,  1.19it/s]


Pearson IC = 0.1718
Spearman IC = 0.1891
R^2 (no intercept, standard) = 0.0290
R^2 (no intercept, formula) = 0.0290
MSE = 7.537414


100%|██████████| 41/41 [00:14<00:00,  2.85it/s]


Pearson IC = 0.1578
Spearman IC = 0.2038
R^2 (no intercept, standard) = 0.0240
R^2 (no intercept, formula) = 0.0240
MSE = 9.272866
